In [2]:
'''
PRAW ONLY GIVES ACCESS TO MOST RECENT POSTS
import praw
from dotenv import load_dotenv
import os

load_dotenv()

client_id = os.getenv("REDDIT_CLIENT_ID")
client_secret = os.getenv("REDDIT_SECRET")
user_agent = os.getenv("REDDIT_USER_AGENT")


reddit = praw.Reddit(
    client_id=client_id,
    client_secret=client_secret,
    user_agent=user_agent
)

for submission in reddit.subreddit("wallstreetbets").new(limit=1000):
    print(submission.title)
'''

'\nPRAW ONLY GIVES ACCESS TO MOST RECENT POSTS\nimport praw\nfrom dotenv import load_dotenv\nimport os\n\nload_dotenv()\n\nclient_id = os.getenv("REDDIT_CLIENT_ID")\nclient_secret = os.getenv("REDDIT_SECRET")\nuser_agent = os.getenv("REDDIT_USER_AGENT")\n\n\nreddit = praw.Reddit(\n    client_id=client_id,\n    client_secret=client_secret,\n    user_agent=user_agent\n)\n\nfor submission in reddit.subreddit("wallstreetbets").new(limit=1000):\n    print(submission.title)\n'

In [3]:
import kagglehub
import os

os.environ['KAGGLEHUB_CACHE'] = os.getcwd() + '\\data'

path = kagglehub.dataset_download("gpreda/reddit-wallstreetsbets-posts")

print("Path to dataset files:", path)

Path to dataset files: C:\Users\jerry\PycharmProjects\StockPredictor\data\datasets\gpreda\reddit-wallstreetsbets-posts\versions\106


In [4]:
import pandas as pd
wsb_posts = pd.read_csv('data\\datasets\\gpreda\\reddit-wallstreetsbets-posts\\versions\\106\\reddit_wsb.csv')
wsb_posts

,title,score,id,url,comms_num,created,body,timestamp
0,"It's not about the money, it's about sending a...",55,l6ulcx,https://v.redd.it/6j75regs72e61,6,1.611863e+09,NaN,2021-01-28 21:37:41
1,Math Professor Scott Steiner says the numbers ...,110,l6uibd,https://v.redd.it/ah50lyny62e61,23,1.611862e+09,NaN,2021-01-28 21:32:10
2,Exit the system,0,l6uhhn,https://www.reddit.com/r/wallstreetbets/commen...,47,1.611862e+09,The CEO of NASDAQ pushed to halt trading “to g...,2021-01-28 21:30:35
3,NEW SEC FILING FOR GME! CAN SOMEONE LESS RETAR...,29,l6ugk6,https://sec.report/Document/0001193125-21-019848/,74,1.611862e+09,NaN,2021-01-28 21:28:57
4,"Not to distract from GME, just thought our AMC...",71,l6ufgy,https://i.redd.it/4h2sukb662e61.jpg,156,1.611862e+09,NaN,2021-01-28 21:26:56
...,...,...,...,...,...,...,...,...
53182,What I Learned Investigating SAVA FUD Spreaders,238,owd2pn,https://www.reddit.com/r/wallstreetbets/commen...,87,1.627906e+09,***TLDR: Three bitter scientists partnered up ...,2021-08-02 15:03:27
53183,"Daily Popular Tickers Thread for August 02, 20...",228,owd1a5,https://www.reddit.com/r/wallstreetbets/commen...,1070,1.627906e+09,\nYour daily hype thread. Please keep the shit...,2021-08-02 15:01:03
53184,Hitler reacts to the market being irrational,7398,owc5dr,https://v.redd.it/46jxu074exe71,372,1.627902e+09,NaN,2021-08-02 13:59:35
53185,"Daily Discussion Thread for August 02, 2021",338,owbfjf,https://www.reddit.com/r/wallstreetbets/commen...,11688,1.627898e+09,Your daily trading discussion thread. Please k...,2021-08-02 13:00:16


In [5]:
from transformers import pipeline
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

sentiment_analyzer = pipeline(
    "sentiment-analysis",
    model="distilbert-base-uncased-finetuned-sst-2-english",
    device=0
)

batch_size = 2048

def batch_sentiment_analysis(texts, batch_size=batch_size):
    results = []
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        batch_results = sentiment_analyzer(batch)
        results.extend(batch_results)
    return results

sentiments = batch_sentiment_analysis(wsb_posts['title'].tolist(), batch_size=batch_size)
wsb_posts['sentiment'] = [res['label'] for res in sentiments]
wsb_posts['sentiment_score'] = [res['score'] for res in sentiments]

embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = embedding_model.encode(
    wsb_posts['title'].tolist(), batch_size=64, show_progress_bar=True
)

dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(np.array(embeddings))  # Add all vectors

faiss.write_index(index, "wsb_index.faiss")
wsb_posts.to_csv("wsb_metadata.csv", index=False)


index = faiss.read_index("wsb_index.faiss")
wsb_posts = pd.read_csv("wsb_metadata.csv")
query = "Should I invest in AMC?"
query_vec = embedding_model.encode([query])

D, I = index.search(np.array(query_vec), k=5)

print("Top semantic matches for query:")
for idx in I[0]:
    row = wsb_posts.iloc[idx]
    print(f"- {row['title']} ({row['sentiment']}, score={row['sentiment_score']:.2f})")


Device set to use cuda:0
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

C:\Users\jerry\PycharmProjects\StockPredictor\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\jerry\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/832 [00:00<?, ?it/s]

Top semantic matches for query:
- Should I buy AMC now? (NEGATIVE, score=1.00)
- Should I buy AMC now? I’m unsure. (NEGATIVE, score=1.00)
- Should I buy more $AMC? (NEGATIVE, score=1.00)
- Is $AMC coming back shall I invest dudes (NEGATIVE, score=0.99)
- Guys is it worth investing in amc stock now??? (NEGATIVE, score=0.94)
